In [42]:
%pip install pymongo
%pip install faker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [43]:
import csv
from pymongo import MongoClient

import random
from faker import Faker

fake = Faker()

In [44]:
client = MongoClient("mongodb://localhost:27017/")
db = client["movie_database"] 

In [45]:
#Create movie collection
movie_collection = db["movie"]

# Read movies data CSV and insert into movie collection
with open('movies_data.csv', mode='r', encoding='latin1') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Create the movie document
        movie = {
            "title": row['Movie'],
            "director": row['Director'],
            "actors": [row['Actor 1'], row['Actor 2'], row['Actor 3']],
            "genre": row['Genre'],
            "budget": int(row['Budget']),
            "release_year": int(row['Release year']),
            "imdb_score": float(row['IMDb score']),
            "description": "N/A"  # Placeholder for description, will be updated later
        }
        movie_collection.insert_one(movie)

#set description
with open('imdb_top_1000.csv', mode='r', encoding='latin1') as file:
    reader = csv.DictReader(file)
    for row in reader:
        movie_collection.update_one(
            {"title": row['Series_Title']},
            {"$set": {
                "description": row['Overview'],
            }}
        )

In [46]:
#Create movie performance collection
movie_performance_collection = db["movie_performance"]

with open('movies_data.csv', mode='r', encoding='latin1') as file:
    reader = csv.DictReader(file)
    for row in reader:
        oscars_and_golden_globes_awards = row['Oscar and Golden Globes awards'].strip()
        if oscars_and_golden_globes_awards == '':
            oscars_and_golden_globes_awards = 0
        else:
            oscars_and_golden_globes_awards = int(oscars_and_golden_globes_awards)

        # Find the movie by title in the movie collection
        movie = movie_collection.find_one({"title": row['Movie']})
        
        if movie:  # If the movie exists in the movie collection
            movie_id = movie['_id']  # Get the movie's ObjectId
            
            # Create the movie performance document
            movie_performance = {
                "movie_id": movie_id,  # Link to the movie collection by ObjectId
                "final_box_office": int(row['Box Office']),
                "earnings": int(row['Earnings']),
                "actor_boxoffice_%": float(row['Actors Box Office %']),
                "director_boxoffice_%": float(row['Director Box Office %']),
                "oscars_and_golden_globes_nominations": int(row['Oscar and Golden Globes nominations']),
                "oscars_and_golden_globes_awards": oscars_and_golden_globes_awards,
                "performance": "N/A"  # performance that difference between predict and actual
            }

            movie_performance_collection.insert_one(movie_performance)

In [47]:
#Create user collection
user_collection = db["users"]

user_data_list = []
for _ in range(10):
    registration_date = fake.date_this_decade()
    last_login = fake.date_this_month()

    # Convert dates to strings in the format "YYYY-MM-DD"
    registration_date_str = registration_date.strftime("%Y-%m-%d")
    last_login_str = last_login.strftime("%Y-%m-%d")

    user_data = {
        "username": fake.user_name(),
        "email": fake.email(),
        "full_name": fake.name(),
        "registration_date": registration_date_str,
        "last_login": last_login_str,
        "preferred_language": random.choice(["English", "French", "Spanish", "German", "Italian", "Japanese", "Korean", "Portuguese", "Chinese"])
    }
    user_data_list.append(user_data)

# Insert the user data into the collection
user_collection.insert_many(user_data_list)

print("fake user records inserted successfully!")

fake user records inserted successfully!


In [48]:
#Create feature usage collection
feature_usage_collection = db["feature_usage"]

feature_usage_data_list = []
for _ in range(10):
    user_id = random.choice(user_data_list)['username']  # Choose a random user from the user data
    feature_name = random.choice(["box_office_prediction", "movie_trends", "earnings_analysis", "genre_insight", "release_year_comparison"])
    interaction_date = fake.date_this_year()  # Generate a random date within this year
    interaction_date_str = interaction_date.strftime("%Y-%m-%d")# Convert the date to string format (e.g., '2024-05-10')
    interaction_details = f"User interacted with {feature_name} to analyze movies."
    
    # Prediction outcome, could be "success", "failure", or "N/A"
    prediction_outcome = random.choice(["success", "failure", "N/A"])
    
    feature_usage_data = {
        "user_id": user_id,
        "feature_name": feature_name,
        "interaction_date": interaction_date_str,
        "interaction_details": interaction_details,
        "prediction_outcome": prediction_outcome
    }
    feature_usage_data_list.append(feature_usage_data)

feature_usage_collection.insert_many(feature_usage_data_list)

InsertManyResult([ObjectId('674c02e483e2d79fa3fd8997'), ObjectId('674c02e483e2d79fa3fd8998'), ObjectId('674c02e483e2d79fa3fd8999'), ObjectId('674c02e483e2d79fa3fd899a'), ObjectId('674c02e483e2d79fa3fd899b'), ObjectId('674c02e483e2d79fa3fd899c'), ObjectId('674c02e483e2d79fa3fd899d'), ObjectId('674c02e483e2d79fa3fd899e'), ObjectId('674c02e483e2d79fa3fd899f'), ObjectId('674c02e483e2d79fa3fd89a0')], acknowledged=True)

In [49]:
user_collection.update_many({}, {"$set": {"role": "user"}})

UpdateResult({'n': 10, 'nModified': 10, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)